# Fábrica de Snapshots (Snapshot Factory)

Este notebook foi desenhado para rodar no **Google Colab** (com GPU).
Ele cria a estrutura de pastas recomendada e treina modelos Generalistas e Especialistas em lote.

> **Dica Colab**: Descomente a linha de instalação apropriada (Pública ou Privada) abaixo.

In [ ]:
# 1. Setup Ambiental e Instalação
import os
import sys

# --- INSTALAÇÃO (Descomente uma das opções) ---

# Opção A: Repositório PÚBLICO (Mais simples)
# !pip install git+https://github.com/aretw0/predicoes-loterias-caixa.git

# Opção B: Repositório PRIVADO (Requer Token)
# ... (mesmo código de token das secrets) ...

import pandas as pd

# --- LÓGICA DE IMPORTAÇÃO HÍBRIDA ---
# Tenta importar do pacote instalado. Se falhar e estivermos localmente, adiciona o path.
try:
    from src.loterias.megasena import MegaSena
    print("✅ Módulos importados do pacote instalado (pip).")
except ImportError:
    # Fallback para execução local sem 'pip install -e .'
    current_dir = os.getcwd()
    if os.path.exists(os.path.join(current_dir, 'src')):
        sys.path.append(current_dir)
        try:
            from src.loterias.megasena import MegaSena
            from src.loterias.models import TransformerModel, CatBoostModel, LSTMModel
            print("⚠️ Módulos importados via PATH Local (Fallback).")
            print("    Dica: Para evitar isso, rode 'pip install -e .' no seu ambiente local.")
        except ImportError as e:
            print(f"❌ Erro Crítico: Módulos não encontrados nem localmente. {e}")
    else:
        print("❌ Erro: Pacote não instalado e pasta 'src' não encontrada.")

In [ ]:
# 2. Criar Estrutura de Pastas
games = ['megasena', 'lotofacil', 'quina']
types = ['generalistas', 'especialistas/virada', 'especialistas/acumulados']

for game in games:
    for t in types:
        path = f"snapshots/{game}/{t}"
        os.makedirs(path, exist_ok=True)
        print(f"Criado: {path}")

In [ ]:
# 3. Carregar Dados Reais
lottery = MegaSena()
df_full = lottery.preprocess_data()
print(f"Total Concursos: {len(df_full)}")

## 4. Treinar Generalistas (Todos os dados)
Use parâmetros agressivos aqui.

In [ ]:
print("--- Treinando Transformer Generalista ---")
model_gen = TransformerModel(1, 60, 6)
# High-Compute Params
history = model_gen.train(df_full, epochs=100, batch_size=64, 
                          head_size=256, num_heads=4, ff_dim=256)

save_path = "snapshots/megasena/generalistas/transformer_heavy_v1.keras"
model_gen.save(save_path)
print(f"Salvo: {save_path}")

## 5. Treinar Especialistas (Mega da Virada / Final 0 ou 5)
Filtramos o DataFrame para conter apenas concursos com final 0 ou 5.

In [ ]:
# Filtrar apenas finais 0 ou 5
# Assumindo que o índice do DF ou uma coluna 'concurso' ajude, ou inferindo pelo row count se for sequencial.
# O DataManager atual não expõe explicito o numero do concurso como coluna garantida em todos, 
# mas 'preprocess_data' da MegaSena geralmente mantem a ordem.
# Vamos filtrar onde (index + 1) % 5 == 0 (Aproximação, ideal é ter coluna Concurso)

# Se o CSV tiver coluna 'Concurso', usamos ela.
if 'Concurso' in df_full.columns:
    df_virada = df_full[df_full['Concurso'] % 5 == 0].copy()
else:
    # Fallback: Mega Sena é sequencial. O índice do dataframe geralmente alinha se não houver buracos.
    print("Aviso: Coluna 'Concurso' não encontrada. Usando índice.")
    df_virada = df_full[df_full.index % 5 == 0].copy() # Simplificação

print(f"Dados Filtrados (Virada/Especiais): {len(df_virada)} concursos.")

print("--- Treinando Transformer Especialista (Virada) ---")
model_spec = TransformerModel(1, 60, 6)
# Treinamos com MENOS épocas talvez, pois tem menos dados, ou MAIS para overfitar o padrão?
# Especialistas podem se dar ao luxo de 'decorar' mais.
model_spec.train(df_virada, epochs=200, batch_size=16) 

save_path_spec = "snapshots/megasena/especialistas/virada/transformer_virada_v1.keras"
model_spec.save(save_path_spec)
print(f"Salvo: {save_path_spec}")

In [ ]:
# 6. Baixar Snapshots (Google Colab)
# Compacta e baixa a pasta de snapshots automaticamente
import shutil
try:
    from google.colab import files
    shutil.make_archive('snapshots_backup', 'zip', 'snapshots')
    files.download('snapshots_backup.zip')
except ImportError:
    print("Ambiente não é o Colab ou google.colab não encontrado.")